In [3]:
import numpy as np
import matplotlib.pyplot as plt

dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"
# load matlab files "S02_restingPre_EC.mat", "S02_restingPre_EO.mat" and so on
# and save them as numpy files
from scipy.io import loadmat

for i in [10,11]:
    for j in ["EC", "EO"]:
        data = loadmat(f"{dir}/S{i}_restingPre_{j}.mat")
        np.save(f"{dir}/S{i}_restingPre_{j}.npy", data)
        print(f"{dir}/S{i}_restingPre_{j}.npy saved")



C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/S10_restingPre_EC.npy saved
C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/S10_restingPre_EO.npy saved
C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/S11_restingPre_EC.npy saved
C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/S11_restingPre_EO.npy saved


In [19]:
# load numpy files
import numpy as np
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"

# we have 10 patients, each with 2 conditions, load all of them
data = {}
for i in range(2,12):
    for j in ["EC", "EO"]:
        data[f"S{i}_{j}"] = np.load(f"{dir}/S0{i}_restingPre_{j}.npy", allow_pickle=True).item()



In [20]:
# save the data as a pickle file
import pickle
with open(f"{dir}/data.pkl", "wb") as f:
    pickle.dump(data, f)
    print(f"{dir}/data.pkl saved")




C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/data.pkl saved


In [53]:
# load the data from the pickle file

import pickle
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"
with open(f"{dir}/data.pkl", "rb") as f:
    data = pickle.load(f)

# print the keys of the data dictionary
print(data.keys())

dict_keys(['S2_EC', 'S2_EO', 'S3_EC', 'S3_EO', 'S4_EC', 'S4_EO', 'S5_EC', 'S5_EO', 'S6_EC', 'S6_EO', 'S7_EC', 'S7_EO', 'S8_EC', 'S8_EO', 'S9_EC', 'S9_EO', 'S10_EC', 'S10_EO', 'S11_EC', 'S11_EO'])


In [54]:
#remove all keys that are not "dataRest"
for key in list(data.keys()):
    for key2 in list(data[key].keys()):
        if key2 != "dataRest":
            del data[key][key2]
print(data["S2_EC"].keys())

dict_keys(['dataRest'])


In [55]:
# we have 68 channels, remove channel 65,66,67
for key in data.keys():
    data[key]["dataRest"] = np.delete(data[key]["dataRest"], [64,65,66], axis=0)


In [75]:
for key in data.keys():
        # replace label values with 0 if it is smaller 210, else set it to 1
        data[key]["dataRest"][-1] = np.where(data[key]["dataRest"][-1] < 210, 0, 1)
        



[1. 1. 1. ... 1. 1. 1.]


In [77]:
# save data as a pickle file
with open(f"{dir}/data.pkl", "wb") as f:
    pickle.dump(data, f)
    print(f"{dir}/data.pkl saved")

    

C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/data.pkl saved


In [152]:
import pickle
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"
with open(f"{dir}/data.pkl", "rb") as f:
    data = pickle.load(f)


In [153]:
data["S2_EC"]["dataRest"].shape
print(38401/256)

150.00390625


In [154]:
# We have 150 seconds of data, with 256 samples per second, create 150 windows of 256 samples
# each window is 1 second of data

windows = []
for key in data.keys():
    for i in range(0, 38000, 256):
        windows.append(data[key]["dataRest"][:,i:i+256])

windows = np.array(windows)
windows.shape






(2980, 65, 256)

In [164]:
import numpy as np
import pandas as pd

# Assuming `data_matrix` is your 3D data of shape (2980, 65, 256)
data_matrix = windows
# Step 1: Extract the data and labels
# Separate the labels (last column in the 65th channel)
labels = data_matrix[:, -1]  # Assuming labels are constant across all time points per window
#print(labels[145:155])

# Step 2: Reshape the data without the labels
data_without_labels = data_matrix[:, :-1, :]  # Exclude the 65th channel (labels)

# Reshape to (2980 * 256, 64)
data_reshaped = data_without_labels.reshape(-1, data_without_labels.shape[1])

# Step 3: Replicate the labels for each time point in the new rows
# Repeat each label 256 times (once for each time point)


# # Step 4: Combine the reshaped data and the repeated labels into a new DataFrame
df = pd.DataFrame(data_reshaped, columns=[f'channel_{i}' for i in range(1, 65)])
df['label'] = labels.reshape(-1)  # Add the repeated labels as a new column

# # Check the shape of the final DataFrame
print(df.shape)  # Should be (762880, 65)
df.head()

# Looks good
print(df["label"][145*256:155*256])
df["label"].value_counts()







(762880, 65)
37120    1.0
37121    1.0
37122    1.0
37123    1.0
37124    1.0
        ... 
39675    0.0
39676    0.0
39677    0.0
39678    0.0
39679    0.0
Name: label, Length: 2560, dtype: float64


1.0    381440
0.0    381440
Name: label, dtype: int64

In [165]:
# save the dataframe as a pickle file
with open(f"{dir}/dataframe.pkl", "wb") as f:
    pickle.dump(df, f)
    print(f"{dir}/dataframe.pkl saved")

    

C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/dataframe.pkl saved


In [167]:
# load the dataframe from the pickle file
import pickle
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"
with open(f"{dir}/dataframe.pkl", "rb") as f:
    df = pickle.load(f)



      channel_1     channel_2     channel_3     channel_4     channel_5  \
0  1.412883e+07  1.412985e+07  1.412550e+07  1.412387e+07  1.412431e+07   
1  1.410380e+07  1.410847e+07  1.409964e+07  1.410329e+07  1.408847e+07   
2  1.410851e+07  1.411286e+07  1.411599e+07  1.411535e+07  1.411033e+07   
3  1.411343e+07  1.411062e+07  1.410969e+07  1.410703e+07  1.411014e+07   
4 -4.909065e+06 -4.907337e+06 -4.912073e+06 -4.920873e+06 -4.917001e+06   

      channel_6     channel_7     channel_8     channel_9    channel_10  ...  \
0  1.412505e+07  1.412047e+07  1.411859e+07  1.412438e+07  1.412313e+07  ...   
1  1.409532e+07  1.410892e+07  1.411276e+07  1.411596e+07  1.410543e+07  ...   
2  1.410796e+07  1.410323e+07  1.410652e+07  1.410252e+07  1.409711e+07  ...   
3  1.410345e+07  1.411078e+07  1.410931e+07  1.411731e+07  1.411807e+07  ...   
4 -4.908233e+06 -4.925417e+06 -4.918121e+06 -4.917065e+06 -4.916745e+06  ...   

     channel_56    channel_57    channel_58    channel_59    channel